# **Importing the necessary Libraries**

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,CuDNNLSTM,Dropout,Dense,LSTM
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.layers import Embedding
import sklearn
import nltk
from sklearn.model_selection import train_test_split
nltk.download("punkt")
from gensim.models import Word2Vec,KeyedVectors;

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#**Loading the Data** 

In [0]:
data = pd.read_csv("All-seasons.csv")
# print(data.head())
lines = data["Line"]
# print(lines.head())

# **Cleaning the Text**

In [0]:
import re

def clean(text):
  text = text.lower()   
  text = re.sub(r"\n", "",  text)
  text = re.sub(r"[-()]", "", text)
  text = re.sub(r"\.", " .", text)
  text = re.sub(r"\!", " !", text)
  text = re.sub(r"\?", " ?", text)
  text = re.sub(r"\,", " ,", text)
  text = re.sub(r"i'm", "i am", text)
  text = re.sub(r"he's", "he is", text)
  text = re.sub(r"she's", "she is", text)
  text = re.sub(r"it's", "it is", text)
  text = re.sub(r"that's", "that is", text)
  text = re.sub(r"what's", "that is", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"won't", "will not", text)
  text = re.sub(r"can't", "cannot", text)
  text = re.sub(r"n't", " not", text)
  text = re.sub(r"n'", "ng", text)
  text = re.sub(r"ohh", "oh", text)
  text = re.sub(r"ohhh", "oh", text)
  text = re.sub(r"ohhhh", "oh", text)
  text = re.sub(r"ohhhhh", "oh", text)
  text = re.sub(r"ohhhhhh", "oh", text)
  text = re.sub(r"ahh", "ah", text)
  return text
clean_text = []
for i in lines:
    clean_text.append(clean(i))

# **Preprocessing the text(converting words to vectors)**

In [0]:
tokenized_text = []
for line in clean_text:
  tokenized_text.append(nltk.word_tokenize(line))
tokenized_text = np.array(tokenized_text)

In [0]:
# word embedding
word_embedding_model = Word2Vec(tokenized_text,min_count=1,size=300)

In [7]:
tokenized_vector = []
for sentences in tokenized_text:
  sentence_vector = [word_embedding_model[w] for w in sentences]
  tokenized_vector.append(sentence_vector)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
padding_vector = np.zeros(300)
# print(padding_vector.shape)

#function to remove list/sentences above 30 words
def max30(listy):
  if len(listy) > 30:
    return False
  else:
    return True

#function to pad lists/sentences with (300,1) zero array if len is smaller than 30
def pad30(listy):
  if len(listy)!=30:
    diff = 30-len(listy)
    for i in range(diff):
      listy.append(padding_vector)
    return np.array(listy)
  else:
    return np.array(listy)

short_vectors = []
short_vectors = list(filter(max30,tokenized_vector))

#padded vectors is a (n,30,300) array
padded_vectors = np.array(list(map(pad30,short_vectors)),dtype=np.float64)

# **Test train Split**

In [0]:
vec_x = padded_vectors[1:]
vec_y = padded_vectors[:-1]
x_train,x_test,y_train,y_test = train_test_split(vec_x,vec_y,test_size=0.2,random_state=1)

# **Model Training and Testing**

In [17]:
model=Sequential()
model.add(LSTM(300,input_shape=x_train.shape[1:],return_sequences=True, activation='sigmoid'))
model.add(LSTM(300,input_shape=x_train.shape[1:],return_sequences=True, activation='sigmoid'))
model.add(LSTM(300,input_shape=x_train.shape[1:],return_sequences=True, activation='sigmoid'))
model.add(LSTM(300,input_shape=x_train.shape[1:],return_sequences=True, activation='sigmoid'))
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.fit(x_train, y_train, nb_epoch=2000,validation_data=(x_test, y_test))
predict = model.predict(x_test)
c = [word_embedding_model.most_similar([predict[10][i]])[0] for i in range(30)]

# **User interacton**

In [0]:
input_sentence = input("enter an sentence")
input_tokenized = nltk.tokenize(input_sentence)
embedded_input = [word_embedding_model[w] for w in input_tokenized]
if len(embedded_input)>30:
  embedded_input[29:] = [] 
elif len(embedded_input)<30:
  diff = 30-len(embedded_input)
  for i in range(diff):
    embedded_input.append(padding_vector)
embedded_input = embedded_input.reshape(1,30,300)
predicted = model.predic(embedded_input)
output = [word_embedding_model.most_similar(i) for i in predicted]
output_string = " ".join(output)